In [1]:
import os
import sys
import time
import datetime
import argparse
import pickle
from skimage.data import imread
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

sys.path.append('/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3')



In [2]:
%load_ext autoreload
%autoreload 2
from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

In [3]:
args = ['--image_folder','../../data',
        '--model_config_path','/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3/config/classifier.cfg',
        '--weights_path','/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3/weights/classifier.weights',
        '--checkpoint_dir','/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3/checkpoints',
        '--img_size','64',
        '--use_cuda','False']
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=30, help='number of epochs')
parser.add_argument('--image_folder', type=str, default='data/samples', help='path to dataset')
parser.add_argument('--batch_size', type=int, default=16, help='size of each image batch')
parser.add_argument('--model_config_path', type=str, default='config/yolov3.cfg', help='path to model config file')
#parser.add_argument('--data_config_path', type=str, default='config/coco.data', help='path to data config file')
parser.add_argument('--weights_path', type=str, default='weights/yolov3.weights', help='path to weights file')
#parser.add_argument('--class_path', type=str, default='data/coco.names', help='path to class label file')
#parser.add_argument('--conf_thres', type=float, default=0.8, help='object confidence threshold')
#parser.add_argument('--nms_thres', type=float, default=0.4, help='iou thresshold for non-maximum suppression')
parser.add_argument('--n_cpu', type=int, default=0, help='number of cpu threads to use during batch generation')
parser.add_argument('--img_size', type=int, default=416, help='size of each image dimension')
parser.add_argument('--checkpoint_interval', type=int, default=1, help='interval between saving model weights')
parser.add_argument('--checkpoint_dir', type=str, default='checkpoints', help='directory where model checkpoints are saved')
parser.add_argument('--use_cuda', type=bool, default=True, help='whether to use cuda if available')
opt = parser.parse_args(args)
print(opt)

Namespace(batch_size=16, checkpoint_dir='/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3/checkpoints', checkpoint_interval=1, epochs=30, image_folder='../../data', img_size=64, model_config_path='/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3/config/classifier.cfg', n_cpu=0, use_cuda=True, weights_path='/Users/mike/Desktop/airbus_kaggle/src/PyTorch-YOLOv3/weights/classifier.weights')


In [4]:
cuda = torch.cuda.is_available() and opt.use_cuda

os.makedirs('output', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)

#classes = load_classes(opt.class_path)

# Get data configuration
#data_config     = parse_data_config(opt.data_config_path)
#train_path      = data_config['train']

# Get hyper parameters
hyperparams     = parse_model_config(opt.model_config_path)[0]
learning_rate   = float(hyperparams['learning_rate'])
momentum        = float(hyperparams['momentum'])
decay           = float(hyperparams['decay'])
burn_in         = int(hyperparams['burn_in'])

# Initiate model
model = Darknet_Classifier(opt.model_config_path)
#model.load_weights(opt.weights_path)
model.apply(weights_init_normal)

if cuda:
    model = model.cuda()

model.train()



Darknet_Classifier(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): BatchNorm2d(64, eps=1

In [5]:
# Get dataloader
outfile = '../../data/box_data.pickle'

with open(outfile, 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    data = pickle.load(f)
    
dataloader = torch.utils.data.DataLoader(
    BoxedDataset('../../data/train',(768,768),data),
    batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, dampening=0, weight_decay=decay)



In [6]:
for epoch in range(opt.epochs):
    for batch_i, (imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)

        optimizer.zero_grad()

        loss = model(imgs, targets)

        loss.backward()
        optimizer.step()

        print('[Epoch %d/%d, Batch %d/%d] [Losses: %f]' %
                                    (epoch, opt.epochs, batch_i, len(dataloader),loss.item()))

        model.seen += imgs.size(0)

    if epoch % opt.checkpoint_interval == 0:
        model.save_weights('%s/%d.weights' % (opt.checkpoint_dir, epoch))

/Users/mike/anaconda/envs/airbus_ship_detection/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[Epoch 0/30, Batch 0/7004] [Losses: 1.156761]
[Epoch 0/30, Batch 1/7004] [Losses: 0.684073]
[Epoch 0/30, Batch 2/7004] [Losses: 0.585298]
[Epoch 0/30, Batch 3/7004] [Losses: 0.889724]
[Epoch 0/30, Batch 4/7004] [Losses: 0.501493]
[Epoch 0/30, Batch 5/7004] [Losses: 0.861910]
[Epoch 0/30, Batch 6/7004] [Losses: 0.955132]
[Epoch 0/30, Batch 7/7004] [Losses: 0.611942]
[Epoch 0/30, Batch 8/7004] [Losses: 0.550343]
[Epoch 0/30, Batch 9/7004] [Losses: 0.868455]
[Epoch 0/30, Batch 10/7004] [Losses: 1.507759]
[Epoch 0/30, Batch 11/7004] [Losses: 0.927770]
[Epoch 0/30, Batch 12/7004] [Losses: 0.509284]
[Epoch 0/30, Batch 13/7004] [Losses: 0.410397]
[Epoch 0/30, Batch 14/7004] [Losses: 1.141224]
[Epoch 0/30, Batch 15/7004] [Losses: 2.223546]
[Epoch 0/30, Batch 16/7004] [Losses: 2.429125]
[Epoch 0/30, Batch 17/7004] [Losses: 0.730958]
[Epoch 0/30, Batch 18/7004] [Losses: 0.879594]
[Epoch 0/30, Batch 19/7004] [Losses: 1.052087]
[Epoch 0/30, Batch 20/7004] [Losses: 0.627325]
[Epoch 0/30, Batch 21/7

KeyboardInterrupt: 

In [ ]:
1024*16

In [ ]:
targets